In [3]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
!pip3 list

Package            Version
------------------ ------------
certifi            2022.12.7
charset-normalizer 2.1.1
filelock           3.9.0
fsspec             2023.4.0
idna               3.4
Jinja2             3.1.2
MarkupSafe         2.1.3
mpmath             1.3.0
networkx           3.0
numpy              1.24.1
Pillow             9.3.0
pip                23.3
requests           2.28.1
setuptools         68.0.0
sympy              1.12
torch              2.1.1+cu118
torchaudio         2.1.1+cu118
torchvision        0.16.1+cu118
typing_extensions  4.4.0
urllib3            1.26.13
wheel              0.41.2


In [1]:
import torchvision

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models import mobilenet_v2

In [4]:
# 转换器
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# 添加标签噪声的函数
def add_label_noise(labels, noise_type='symmetric', noise_rate=0.2):
    noisy_labels = labels.copy()
    n_samples = labels.shape[0]
    n_classes = 10

    if noise_type == 'symmetric':
        n_noisy = int(noise_rate * n_samples)
        noise_idx = np.random.choice(n_samples, n_noisy, replace=False)
        for idx in noise_idx:
            original_label = labels[idx]
            noisy_labels[idx] = np.random.choice([l for l in range(n_classes) if l != original_label])
    elif noise_type == 'pairflip':
        for i in range(n_classes // 2):
            flip_from = i * 2
            flip_to = flip_from + 1
            flip_idx = np.where(labels == flip_from)[0]
            n_flip = int(noise_rate * len(flip_idx))
            flip_idx = np.random.choice(flip_idx, n_flip, replace=False)
            noisy_labels[flip_idx] = flip_to

    return noisy_labels

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [9]:
# MobileNet 模型
net = mobilenet_v2(pretrained=False)
net.classifier[1] = nn.Linear(net.classifier[1].in_features, 10)
net.to(device)
print("start")

# 训练模型
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

for epoch in range(50):  # 遍历数据集多次
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #labels = labels.numpy()
        labels = labels.cpu().numpy()
        # 添加噪声
        noisy_labels = add_label_noise(labels, noise_type='symmetric', noise_rate=0.4)
        noisy_labels = torch.from_numpy(noisy_labels).to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, noisy_labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
         # 打印当前进度
        if i % 200 == 199:    # 每 200 批次打印一次
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}/{len(trainloader)}, Loss: {running_loss / 200}')
            running_loss = 0.0
            

    print(f'Epoch {epoch + 1} loss: {running_loss / len(trainloader)}')

print('Finished Training')

# 评估模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

start
Epoch: 1, Batch: 200/782, Loss: 2.364282908439636
Epoch: 1, Batch: 400/782, Loss: 2.31845291018486
Epoch: 1, Batch: 600/782, Loss: 2.2762377667427063
Epoch 1 loss: 0.5281820876519089
Epoch: 2, Batch: 200/782, Loss: 2.2485444927215577
Epoch: 2, Batch: 400/782, Loss: 2.2461135244369506
Epoch: 2, Batch: 600/782, Loss: 2.2340430134534834
Epoch 2 loss: 0.5137156909688965
Epoch: 3, Batch: 200/782, Loss: 2.206979848742485
Epoch: 3, Batch: 400/782, Loss: 2.1875879365205764
Epoch: 3, Batch: 600/782, Loss: 2.182610951066017
Epoch 3 loss: 0.5060442614433406
Epoch: 4, Batch: 200/782, Loss: 2.164680314064026
Epoch: 4, Batch: 400/782, Loss: 2.1573827505111693
Epoch: 4, Batch: 600/782, Loss: 2.1491634953022003
Epoch 4 loss: 0.4990552788805169
Epoch: 5, Batch: 200/782, Loss: 2.1254815834760667
Epoch: 5, Batch: 400/782, Loss: 2.118248967528343
Epoch: 5, Batch: 600/782, Loss: 2.114011978507042
Epoch 5 loss: 0.4932973499188338
Epoch: 6, Batch: 200/782, Loss: 2.109225798845291
Epoch: 6, Batch: 400/7